In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

#NLP libraries 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/muhammadkamdar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muhammadkamdar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
chunk = pd.read_csv('Bitcoin_tweets.csv',chunksize=100000,lineterminator='\n')
df = pd.concat(chunk)

/var/folders/cy/wq8cw_n951db1z5xgm5fdjz00000gn/T/ipykernel_67025/915893768.py:2: DtypeWarning: Columns (5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunk)
/var/folders/cy/wq8cw_n951db1z5xgm5fdjz00000gn/T/ipykernel_67025/915893768.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunk)
/var/folders/cy/wq8cw_n951db1z5xgm5fdjz00000gn/T/ipykernel_67025/915893768.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunk)
/var/folders/cy/wq8cw_n951db1z5xgm5fdjz00000gn/T/ipykernel_67025/915893768.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunk)
/var/folders/cy/wq8cw_n951db1z5xgm5fdjz00000gn/T/ipykernel_67025/915893768.py:2: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low

In [4]:
df.shape

(4689354, 13)

In [5]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [6]:
df.dropna(axis=0 ,subset=['date','text'],inplace = True)
df.reset_index(drop= True,inplace=True)
df.shape

(4689354, 13)

In [7]:
df.drop_duplicates(inplace = True)
df.reset_index(drop=True,inplace=True)
df.shape

(4689354, 13)

In [8]:
df = df.convert_dtypes()
df.dtypes

user_name            string
user_location        string
user_description     string
user_created         string
user_followers       object
user_friends         object
user_favourites      object
user_verified        object
date                 string
text                 string
hashtags             string
source               string
is_retweet          boolean
dtype: object

In [9]:
def validate(date_text):
    errors = []
    for i,v in enumerate(date_text):
        try:
            datetime.strptime(v, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            errors.append(i)
    return errors

In [10]:
errors = validate(df['date'])
print('There are '+ str(len(errors)) + ' values that are not actual dates in the date column')

There are 66 values that are not actual dates in the date column


In [11]:
df_date_clean = df.loc[~df.index.isin(errors)]
df_date_clean.reset_index(drop=True,inplace=True)
df_date_clean.shape

(4689288, 13)

In [12]:
df_date_clean['date'] = pd.to_datetime(df_date_clean['date'], format='%Y-%m-%d %H:%M:%S')

/var/folders/cy/wq8cw_n951db1z5xgm5fdjz00000gn/T/ipykernel_67025/3605006481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_clean['date'] = pd.to_datetime(df_date_clean['date'], format='%Y-%m-%d %H:%M:%S')


In [13]:
df_date_clean_final = df_date_clean.loc[(df_date_clean['date'] >= '2022-05-30')].reset_index(drop=True)
df_date_clean_final.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:49,$1 billion has been lost in cryptocurrency sca...,"['trm', 'trm_crypto', 'Bitcoin', 'Etherium', '...",IFTTT,False
1,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:49,Is Solana a 'buy' with SOL price at 10-month l...,"['trm', 'trm_crypto', 'Bitcoin', 'Etherium', '...",IFTTT,False
2,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:48,Russia's Industrial Giant Rostec Announces Blo...,"['trm', 'trm_crypto', 'Bitcoin', 'Etherium', '...",IFTTT,False
3,CrypticThreadz.NFT,Metaverse,News collected from multiple top sources in on...,2021-07-17 23:56:22,265.0,81.0,94.0,False,2022-06-04 23:59:33,Alchemy announces support for Solana Web3 appl...,"['bitcoin', 'cryptocurrency', 'crypto', 'nft',...",Blogtotwitter1950,False
4,Freelance Jobs Feed,<NA>,Follow here for Hourly Freelance Job Posts | S...,2011-12-24 00:26:53,13050.0,6.0,0.0,False,2022-06-04 23:59:32,#Bitcoin #Blockchain Blockfriend nft bot: I ne...,"['Bitcoin', 'Blockchain']",dlvr.it,False


In [29]:
df_date_clean_final.shape

(1103654, 13)

In [15]:
irrelevant_hashtags = ['nft','giveaway','cashback','airdrop']
df_date_clean_final['hashtags'] = df_date_clean_final['hashtags'].str.lower()

In [16]:
df_date_clean_final = df_date_clean_final[df_date_clean_final['hashtags'].str.contains('|'.join(irrelevant_hashtags)) == False].reset_index(drop = True)

In [17]:
df_date_clean_final.shape

(1103654, 13)

In [18]:
df_date_clean_final['text'] = df_date_clean_final['text'].str.replace("(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"," ")

/var/folders/cy/wq8cw_n951db1z5xgm5fdjz00000gn/T/ipykernel_67025/2436676272.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_date_clean_final['text'] = df_date_clean_final['text'].str.replace("(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"," ")


In [19]:
df_clean = df_date_clean_final.copy()
df_clean.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:49,1 billion has been lost in cryptocurrency sca...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False
1,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:49,Is Solana a buy with SOL price at 10 month l...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False
2,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:48,Russia s Industrial Giant Rostec Announces Blo...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False
3,Freelance Jobs Feed,<NA>,Follow here for Hourly Freelance Job Posts | S...,2011-12-24 00:26:53,13050.0,6.0,0.0,False,2022-06-04 23:59:32,Bitcoin Blockchain Blockfriend nft bot I ne...,"['bitcoin', 'blockchain']",dlvr.it,False
4,Alex,"Havana, Cuba",#Bitcoin 💎🙌🏻 #CryptoTrader📈📉,2020-12-31 21:48:00,153.0,127.0,7294.0,False,2022-06-04 23:59:25,hfangca Bitcoin is freedom for me It s a wa...,['bitcoin'],Twitter Web App,False


In [20]:
df_clean['text'] = df_clean.apply(lambda row: word_tokenize(row['text']), axis=1)

In [21]:
df_clean.shape

(1103654, 13)

In [22]:
df_clean.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:49,"[1, billion, has, been, lost, in, cryptocurren...","['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False
1,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:49,"[Is, Solana, a, buy, with, SOL, price, at, 10,...","['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False
2,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:48,"[Russia, s, Industrial, Giant, Rostec, Announc...","['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False
3,Freelance Jobs Feed,<NA>,Follow here for Hourly Freelance Job Posts | S...,2011-12-24 00:26:53,13050.0,6.0,0.0,False,2022-06-04 23:59:32,"[Bitcoin, Blockchain, Blockfriend, nft, bot, I...","['bitcoin', 'blockchain']",dlvr.it,False
4,Alex,"Havana, Cuba",#Bitcoin 💎🙌🏻 #CryptoTrader📈📉,2020-12-31 21:48:00,153.0,127.0,7294.0,False,2022-06-04 23:59:25,"[hfangca, Bitcoin, is, freedom, for, me, It, s...",['bitcoin'],Twitter Web App,False


In [23]:
stop_words = set(stopwords.words('english'))
df_clean['text'] = df_clean.apply(lambda row: (" ".join([word for word in row['text'] if word not in stop_words])),axis=1)


In [24]:
df_clean[['polarity', 'subjectivity']] = df_clean['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

In [25]:
df_clean.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,polarity,subjectivity
0,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:49,1 billion lost cryptocurrency scams since 2021...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False,0.000000,0.000000
1,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:49,Is Solana buy SOL price 10 month lows 85 peak ...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False,0.000000,0.000000
2,trm_crypto,United States,This is the trm_crypto channel and I want to b...,2021-11-05 14:08:29,50.0,644.0,348.0,False,2022-06-04 23:59:48,Russia Industrial Giant Rostec Announces Block...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",IFTTT,False,0.000000,1.000000
3,Freelance Jobs Feed,<NA>,Follow here for Hourly Freelance Job Posts | S...,2011-12-24 00:26:53,13050.0,6.0,0.0,False,2022-06-04 23:59:32,Bitcoin Blockchain Blockfriend nft bot I need ...,"['bitcoin', 'blockchain']",dlvr.it,False,0.214286,0.642857
4,Alex,"Havana, Cuba",#Bitcoin 💎🙌🏻 #CryptoTrader📈📉,2020-12-31 21:48:00,153.0,127.0,7294.0,False,2022-06-04 23:59:25,hfangca Bitcoin freedom It way corrupted syste...,['bitcoin'],Twitter Web App,False,-0.066667,0.495833


In [26]:
df_sentiment = df_clean.copy()

In [27]:
df_sentiment['Sentiment'] = df_sentiment["polarity"].apply(lambda x: "Positive" if x > 0 else( "Negative" if x < 0  else "Nuetral"))

In [28]:
df_sentiment.drop(['user_description','user_created','source','polarity','subjectivity'],axis=1,inplace=True)
df_sentiment.head()

,user_name,user_location,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,is_retweet,Sentiment
0,trm_crypto,United States,50.0,644.0,348.0,False,2022-06-04 23:59:49,1 billion lost cryptocurrency scams since 2021...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",False,Nuetral
1,trm_crypto,United States,50.0,644.0,348.0,False,2022-06-04 23:59:49,Is Solana buy SOL price 10 month lows 85 peak ...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",False,Nuetral
2,trm_crypto,United States,50.0,644.0,348.0,False,2022-06-04 23:59:48,Russia Industrial Giant Rostec Announces Block...,"['trm', 'trm_crypto', 'bitcoin', 'etherium', '...",False,Nuetral
3,Freelance Jobs Feed,<NA>,13050.0,6.0,0.0,False,2022-06-04 23:59:32,Bitcoin Blockchain Blockfriend nft bot I need ...,"['bitcoin', 'blockchain']",False,Positive
4,Alex,"Havana, Cuba",153.0,127.0,7294.0,False,2022-06-04 23:59:25,hfangca Bitcoin freedom It way corrupted syste...,['bitcoin'],False,Negative
